In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)
# Uncomment the follwing line to enabled devlib debugging statements
logging.getLogger('ssh').setLevel(logging.DEBUG)

In [2]:
import json
import time
import os

# Test environment setup

In [3]:
# Setup a target configuration
my_target_conf = {
    "platform"    : "linux",
    "board"       : "juno",
    "modules"     : ['bl',"cpufreq"],
    "host"        : "192.168.0.10",
    "username"    : "root",
    "password"    : "",
    "tftp"  : {
        "folder"    : "/var/lib/tftpboot",
        "kernel"    : "kern.bin",
        "dtb"       : "dtb.bin"
    },
}

# Setup the required Test Environment supports
my_tests_conf = {
    # list of additional devlib modules to install 
    "modules" : ['hwmon'],
    # list of additional binary tools to install
    "tools" : ['rt-app', 'taskset'],
    "ftrace" : {
         "events" : [
             "cpu_idle",
             "sched_switch",
         ],
         "buffsize" : 10240
    }
}

In [4]:
from env import TestEnv

# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)

07:16:29  INFO    :         Target - Using base path: /home/derkling/Code/schedtest
07:16:29  INFO    :         Target - Connecting linux target with: {'username': 'root', 'host': '192.168.0.10', 'password': ''}
07:16:29  DEBUG   : Logging in root@192.168.0.10
07:16:31  DEBUG   : echo $PATH
07:16:31  DEBUG   : ls -1 /usr/local/bin
07:16:31  DEBUG   : cat /proc/cpuinfo
07:16:32  DEBUG   : id
07:16:32  DEBUG   : sudo -- sh -c 'dmidecode -s system-version'
07:16:33  DEBUG   : uname -m
07:16:33  DEBUG   : if [ -e '/sys/devices/system/cpu/cpufreq' ]; then echo 1; else echo 0; fi
07:16:34  DEBUG   : if [ -e '/sys/devices/system/cpu/cpu0/cpufreq' ]; then echo 1; else echo 0; fi
07:16:34  DEBUG   : if [ -e '/sys/class/hwmon' ]; then echo 1; else echo 0; fi
07:16:34  DEBUG   : ls -1 /sys/class/hwmon
07:16:35  DEBUG   : if [ -e '/sys/class/hwmon/hwmon0/name' ]; then echo 1; else echo 0; fi
07:16:35  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon0/name'\'''
07:16:36  DEBUG   : ls -1 /sys

## Attributes

The initialization of the test environment pre-initialize some useful<br>
environment variables which are available to write test cases.

These are some of the information available via the TestEnv object.

In [5]:
# The complete configuration of the target we have configured
print json.dumps(te.conf, indent=4)

{
    "username": "root", 
    "ftrace": {
        "buffsize": 10240, 
        "events": [
            "cpu_idle", 
            "sched_switch"
        ]
    }, 
    "modules": [
        "bl", 
        "cpufreq"
    ], 
    "platform": "linux", 
    "host": "192.168.0.10", 
    "board": "juno", 
    "__features__": [], 
    "tftp": {
        "kernel": "kern.bin", 
        "folder": "/var/lib/tftpboot", 
        "dtb": "dtb.bin"
    }, 
    "password": "", 
    "tools": []
}


In [6]:
# Last configured kernel and DTB image
print te.kernel
print te.dtb

None
None


In [7]:
# The IP and MAC address of the target
print te.ip
print te.mac

192.168.0.10
None


In [8]:
# A full platform descriptor
print json.dumps(te.platform, indent=4)

{
    "clusters": {
        "big": [
            1, 
            2
        ], 
        "little": [
            0, 
            3, 
            4, 
            5
        ]
    }, 
    "cpus_count": 6, 
    "freqs": {
        "big": [
            450000, 
            625000, 
            800000, 
            950000, 
            1100000
        ], 
        "little": [
            450000, 
            575000, 
            700000, 
            775000, 
            850000
        ]
    }, 
    "topology": [
        [
            0, 
            3, 
            4, 
            5
        ], 
        [
            1, 
            2
        ]
    ]
}


In [9]:
# A pre-created folder to host the tests results generated using this
# test environment, notice that the suite could add additional information
# in this folder, like for example a copy of the target configuration
# and other target specific collected information
te.res_dir

'/home/derkling/Code/schedtest/results/20151110_191720'

In [10]:
# The working directory on the target
te.workdir

'/data/local/schedtest'

In [11]:
# The target topology, which can be used to build BART assertions
te.topology

## Functions

Some methods are also exposed to test developers which could be used to easy
the creation of tests.

These are some of the methods available:

In [12]:
# Calibrate RT-App (if required) and get the most updated calibration value
te.calibration()

{0: 352, 1: 138, 2: 138, 3: 355, 4: 361, 5: 353}

In [13]:
# Generate a JSON file with the complete platform description
te.platform_dump(dest_dir='/tmp')

In [14]:
# Force a reboot of the target (and wait specified [s] before reconnect)
te.reboot(reboot_time=60)

07:17:22  DEBUG   : sudo -- sh -c 'sleep 2 && reboot -f &'
07:17:23  INFO    :         Reboot - Waiting 60 [s]for target to reboot...
07:18:23  INFO    :         Target - Connecting linux target with: {'username': 'root', 'host': '192.168.0.10', 'password': ''}
07:18:23  DEBUG   : Logging in root@192.168.0.10
07:18:24  DEBUG   : echo $PATH
07:18:24  DEBUG   : ls -1 /usr/local/bin
07:18:25  DEBUG   : cat /proc/cpuinfo
07:18:25  DEBUG   : id
07:18:26  DEBUG   : sudo -- sh -c 'dmidecode -s system-version'
07:18:26  DEBUG   : uname -m
07:18:26  DEBUG   : if [ -e '/sys/devices/system/cpu/cpufreq' ]; then echo 1; else echo 0; fi
07:18:27  DEBUG   : if [ -e '/sys/devices/system/cpu/cpu0/cpufreq' ]; then echo 1; else echo 0; fi
07:18:27  DEBUG   : if [ -e '/sys/class/hwmon' ]; then echo 1; else echo 0; fi
07:18:28  DEBUG   : ls -1 /sys/class/hwmon
07:18:28  DEBUG   : if [ -e '/sys/class/hwmon/hwmon0/name' ]; then echo 1; else echo 0; fi
07:18:29  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwm

In [15]:
# Resolve a MAC address into an IP address
te.resolv_host(host='00:02:F7:00:5A:5B')

07:19:00  INFO    :   HostResolver - Target (00:02:F7:00:5A:5B) at IP address: 192.168.0.10


('00:02:F7:00:5A:5B', '192.168.0.10')

In [16]:
# Copy the specified file into the TFTP server folder defined by configuration
te.tftp_deploy('/etc/group')

07:19:00  INFO    :           TFTP - Deploy /etc/group into /var/lib/tftpboot/group


A special TestEnv attribute is <b>target</b>, which represent a <b>devlib instance</b>.<br>
Using the target attribute we can access to the full set of devlib provided<br>
functionalities. Which are summarized in the following sections.


# Access to the devlib API

In [17]:
# Run a command on the target
te.target.execute("echo -n 'Hello Test Environment'", as_root=False)

07:19:01  DEBUG   : echo -n 'Hello Test Environment'


'Hello Test Environment'

In [18]:
# Spawn a command in background on the target
te.target.kick_off("sleep 10", as_root=True)

07:19:01  DEBUG   : sudo -- sh -c 'sh -c "sleep 10" 1>/dev/null 2>/dev/null &'


''

In [19]:
# Acces to many target specific information
print "ABI                 : ", te.target.abi
print "big Core Family     : ", te.target.big_core
print "LITTLE Core Family  : ", te.target.little_core
print "CPU's Clusters IDs  : ", te.target.core_clusters
print "CPUs type           : ", te.target.core_names

ABI                 :  arm64
big Core Family     :  A57
LITTLE Core Family  :  A53
CPU's Clusters IDs  :  [0, 1, 1, 0, 0, 0]
CPUs type           :  ['A53', 'A57', 'A57', 'A53', 'A53', 'A53']


In [20]:
# Access to big.LITTLE specific information
print "big CPUs IDs        : ", te.target.bl.bigs
print "LITTLE CPUs IDs     : ", te.target.bl.littles
print "big CPUs freqs      : {}".format(te.target.bl.get_bigs_frequency())
print "big CPUs governor   : {}".format(te.target.bl.get_bigs_governor())

07:19:02  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/online'\'''
07:19:02  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu1/cpufreq/scaling_cur_freq'\'''


big CPUs IDs        :  [1, 2]
LITTLE CPUs IDs     :  [0, 3, 4, 5]
big CPUs freqs      : 1100000

07:19:03  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/online'\'''
07:19:03  DEBUG   : sudo -- sh -c 'cat '\''/sys/devices/system/cpu/cpu1/cpufreq/scaling_governor'\'''



big CPUs governor   : performance


# Sample energy from the target

In [26]:
# Reset and sample energy counters
te.emeter.reset()
nrg = te.emeter.sample()
nrg = json.dumps(te.emeter.sample(), indent=4)
print "First read: ", nrg
time.sleep(2)
nrg = te.emeter.sample()
nrg = json.dumps(te.emeter.sample(), indent=4)
print "Second read: ", nrg

07:20:08  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon14/energy1_input'\'''
07:20:09  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon13/energy1_input'\'''
07:20:09  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon14/energy1_input'\'''
07:20:10  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon13/energy1_input'\'''
07:20:10  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon14/energy1_input'\'''
07:20:10  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon13/energy1_input'\'''
07:20:13  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon14/energy1_input'\'''
07:20:13  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon13/energy1_input'\'''
07:20:14  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon14/energy1_input'\'''
07:20:14  DEBUG   : sudo -- sh -c 'cat '\''/sys/class/hwmon/hwmon13/energy1_input'\'''


First read:  {
    "a53": {
        "total": 0.5966060000000013, 
        "last": 57.526971, 
        "delta": 0.29631200000000035
    }, 
    "a57": {
        "total": 0.6072220000000002, 
        "last": 61.852674, 
        "delta": 0.30055699999999774
    }
}
Second read:  {
    "a53": {
        "total": 1.513701999999995, 
        "last": 58.444067, 
        "delta": 0.29879099999999426
    }, 
    "a57": {
        "total": 1.6665810000000008, 
        "last": 62.912033, 
        "delta": 0.3054509999999979
    }
}


# Configure FTrace for a sepcific experiment

In [27]:
# Configure a specific set of events to trace
te.ftrace_conf(
    {                                                                                                                                             
         "events" : [                                                                                                                                            
             "cpu_idle",                                                                                                                                         
             "cpu_capacity",
             "cpu_frequency",
             "sched_switch"
         ],                                                                                                                                                      
         "buffsize" : 10240                                                                                                                                      
    }
)

07:20:32  DEBUG   : /usr/bin/scp -r   /home/derkling/Code/schedtest/libs/devlib/devlib/bin/arm64/trace-cmd root@192.168.0.10:/root/devlib-target/bin/trace-cmd
07:20:32  DEBUG   : chmod a+x /root/devlib-target/bin/trace-cmd
07:20:33  INFO    :         FTrace - Enabled events:
07:20:33  INFO    :         FTrace -   ['cpu_idle', 'cpu_capacity', 'cpu_frequency', 'sched_switch']


In [30]:
# Start/Stop a FTrace session
te.ftrace.start()
time.sleep(2)
te.ftrace.stop()

07:21:25  DEBUG   : sudo -- sh -c 'echo 10240 > '\''/sys/kernel/debug/tracing/buffer_size_kb'\'''
07:21:26  DEBUG   : sudo -- sh -c 'cat '\''/sys/kernel/debug/tracing/buffer_size_kb'\'''
07:21:26  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd reset'
07:21:29  DEBUG   : sudo -- sh -c 'echo TRACE_MARKER_START > '\''/sys/kernel/debug/tracing/trace_marker'\'''
07:21:29  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd start -e cpu_idle -e cpu_capacity -e cpu_frequency -e sched_switch'
07:21:34  DEBUG   : sudo -- sh -c 'echo TRACE_MARKER_STOP > '\''/sys/kernel/debug/tracing/trace_marker'\'''
07:21:34  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd stop'


In [31]:
# Collect and visualuze the collected trace
trace_file = os.path.join(te.res_dir, 'trace.dat')
te.ftrace.get_trace(trace_file)
te.ftrace.view(trace_file)

07:21:37  DEBUG   : sudo -- sh -c '/root/devlib-target/bin/trace-cmd extract -o /root/devlib-target/trace.dat'
07:21:40  DEBUG   : /usr/bin/scp -r   root@192.168.0.10:/root/devlib-target/trace.dat /home/derkling/Code/schedtest/results/20151110_191720/trace.dat
